https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip
    
https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

https://nyc-tlc.s3.amazonaws.com/trip+data/yellow_tripdata_2019-12.csv



#!pip install pyshp
#!pip install descartes
#!pip install SQLAlchemy
https://sqlitebrowser.org/dl/
https://www.sqlitetutorial.net/sqlite-python/sqlite-python-select/


In [83]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import folium

In [16]:
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [18]:
from sqlalchemy import create_engine

nyc_database = create_engine('sqlite:///nyc_database.db')

In [20]:
for month in range(1,2):
    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+ \
                               "yellow_tripdata_2019-{0:0=2d}.csv".format(month), 
                               "nyc.2017-{0:0=2d}.csv".format(month))

In [48]:
# Download the location Data
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./shape")

In [80]:
df = pd.read_csv("data/nyc.2017-01.csv", nrows=10)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [82]:
df.VendorID.unique()

array([1, 2], dtype=int64)

In [25]:
j, chunksize = 1, 100000
for month in range(1,2):
    fp = "nyc.2017-{0:0=2d}.csv".format(month)
    for df in pd.read_csv(fp, chunksize=chunksize, iterator=True):
        df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns})
        df['pickup_hour'] = [x[11:13] for x in df['tpep_pickup_datetime']]
        df['dropoff_hour'] = [x[11:13] for x in df['tpep_dropoff_datetime']]
        df.index += j
        df.to_sql('table_record', nyc_database, if_exists='append')
        j = df.index[-1] + 1
del df

In [37]:
sf = shapefile.Reader("data/shape/taxi_zones.shp")
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['OBJECTID', 'N', 9, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11],
 ['zone', 'C', 254, 0],
 ['LocationID', 'N', 4, 0],
 ['borough', 'C', 254, 0]]

In [28]:
fields_name = [field[0] for field in sf.fields[1:]]
fields_name

['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough']

In [32]:
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()

In [66]:
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]
shp_attr

[{'OBJECTID': 1,
  'Shape_Leng': 0.116357453189,
  'Shape_Area': 0.0007823067885,
  'zone': 'Newark Airport',
  'LocationID': 1,
  'borough': 'EWR'},
 {'OBJECTID': 2,
  'Shape_Leng': 0.43346966679,
  'Shape_Area': 0.00486634037837,
  'zone': 'Jamaica Bay',
  'LocationID': 2,
  'borough': 'Queens'},
 {'OBJECTID': 3,
  'Shape_Leng': 0.0843411059012,
  'Shape_Area': 0.000314414156821,
  'zone': 'Allerton/Pelham Gardens',
  'LocationID': 3,
  'borough': 'Bronx'},
 {'OBJECTID': 4,
  'Shape_Leng': 0.0435665270921,
  'Shape_Area': 0.000111871946192,
  'zone': 'Alphabet City',
  'LocationID': 4,
  'borough': 'Manhattan'},
 {'OBJECTID': 5,
  'Shape_Leng': 0.0921464898574,
  'Shape_Area': 0.000497957489363,
  'zone': 'Arden Heights',
  'LocationID': 5,
  'borough': 'Staten Island'},
 {'OBJECTID': 6,
  'Shape_Leng': 0.150490542523,
  'Shape_Area': 0.000606460984581,
  'zone': 'Arrochar/Fort Wadsworth',
  'LocationID': 6,
  'borough': 'Staten Island'},
 {'OBJECTID': 7,
  'Shape_Leng': 0.1074171711

In [67]:
df = pd.DataFrame(shp_attr)
df

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,1,0.116357,0.000782,Newark Airport,1,EWR
1,2,0.433470,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,0.000498,Arden Heights,5,Staten Island
...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx
259,260,0.133514,0.000422,Woodside,260,Queens
260,261,0.027120,0.000034,World Trade Center,261,Manhattan
261,262,0.049064,0.000122,Yorkville East,262,Manhattan


In [49]:
import geopandas


In [71]:
import geopandas as gpd
latlondf = gpd.read_file('data/NY-Places.geojson')
latlondf = latlondf[["Id", "geometry"]]

In [65]:
latlondf.geometry

0     POINT (-73.88896 40.89621)
1     POINT (-73.87165 40.88988)
2     POINT (-73.83864 40.88697)
3     POINT (-73.80980 40.87799)
4     POINT (-73.87831 40.86442)
                 ...            
91    POINT (-73.91683 40.89231)
92    POINT (-73.86497 40.65914)
93    POINT (-74.00743 40.72660)
94    POINT (-73.96628 40.70052)
95    POINT (-73.93440 40.55099)
Name: geometry, Length: 96, dtype: geometry

In [75]:
df_new = pd.merge(df, latlondf, left_on='LocationID', right_on='Id')
df_new

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,Id,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,1,POINT (-73.74427 40.75366)
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,2,POINT (-73.83062 40.67219)
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,3,POINT (-73.92427 40.77679)
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,4,POINT (-73.78692 40.67325)
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,5,POINT (-73.85499 40.67729)
...,...,...,...,...,...,...,...,...
91,95,0.113630,0.000573,Forest Hills,95,Queens,95,POINT (-73.87165 40.88988)
92,96,0.185180,0.000548,Forest Park/Highland Park,96,Queens,96,POINT (-73.87524 40.84901)
93,96,0.185180,0.000548,Forest Park/Highland Park,96,Queens,96,POINT (-73.96628 40.70052)
94,96,0.185180,0.000548,Forest Park/Highland Park,96,Queens,96,POINT (-73.93440 40.55099)


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,8))
ax = plt.subplot(1, 2, 1)
ax.set_title("Boroughs in NYC")
draw_region_map(ax, sf)
ax = plt.subplot(1, 2, 2)
ax.set_title("Zones in NYC")
draw_zone_map(ax, sf)

In [87]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

folium.CircleMarker(location=[40.738, -73.98],fill=True).add_to(folium_map)
folium_map

KeyError: 0

In [88]:
import sqlite3

In [96]:
query = 'SELECT * from nyc_data limit 10'

In [97]:
db_path = 'nyc_database.db'
conn = sqlite3.connect(db_path)
a_pandas_df = pd.read_sql_query(query, conn)

In [98]:
a_pandas_df

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_hour,dropoff_hour
0,1,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,...,7.0,0.5,0.5,1.65,0.00,0.3,9.95,None,00,00
1,2,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,...,14.0,0.5,0.5,1.00,0.00,0.3,16.30,None,00,01
2,3,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,...,4.5,0.5,0.5,0.00,0.00,0.3,5.80,None,13,13
3,4,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,...,3.5,0.5,0.5,0.00,0.00,0.3,7.55,None,15,15
4,5,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,...,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None,15,15
5,6,2,2018-11-28 16:25:49,2018-11-28 16:28:26,5,0.0,1,N,193,193,...,3.5,0.5,0.5,0.00,5.76,0.3,13.31,None,16,16
6,7,2,2018-11-28 16:29:37,2018-11-28 16:33:43,5,0.0,2,N,193,193,...,52.0,0.0,0.5,0.00,0.00,0.3,55.55,None,16,16
7,8,1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.3,1,N,163,229,...,6.5,0.5,0.5,1.25,0.00,0.3,9.05,None,00,00
8,9,1,2019-01-01 00:32:01,2019-01-01 00:45:39,1,3.7,1,N,229,7,...,13.5,0.5,0.5,3.70,0.00,0.3,18.50,None,00,00
9,10,1,2019-01-01 00:57:32,2019-01-01 01:09:32,2,2.1,1,N,141,234,...,10.0,0.5,0.5,1.70,0.00,0.3,13.00,None,00,01


In [99]:
import findspark
findspark.init()

In [110]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Analyzing London crime data").config('spark.driver.extraClassPath','D:\Programs\spark-3.0.0-preview2-bin-hadoop2.7\bin\sqlite-jdbc-3.30.1.jar').config('spark.executor.extraClassPath','D:\Programs\spark-3.0.0-preview2-bin-hadoop2.7\bin\sqlite-jdbc-3.30.1.jar').getOrCreate()

In [116]:
# spark.driver.extraClassPath = "D:/Programs/spark-3.0.0-preview2-bin-hadoop2.7/bin/sqlite-jdbc-3.30.1.jar"
# spark.executor.extraClassPath = "D:/Programs/spark-3.0.0-preview2-bin-hadoop2.7/bin/sqlite-jdbc-3.30.1.jar"
#sdf = spark.read.format("jdbc").options(url ="jdbc:sqlite:C:/Users/biraj/Documents/Uber Analysis/nyc_database.db", dbtable="nyc_data").load().take(10)

# df = spark.read.format("jdbc").options(url="jdbc:sqlite:C:/Users/biraj/Documents/Uber Analysis/nyc_database.db",driver="com.sqlite.jdbc.Driver",dbtable="nyc_data").load().take(10)